In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ATLAS_style import *
ROOT.gErrorIgnoreLevel = ROOT.kError
set_atlas_style()

In module 'Darwin':
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/OSByteOrder.h:14:1: error: '_OSSwapInt16' has different definitions in different modules; definition in module 'Darwin.libkern.OSByteOrder' first difference is return type is 'uint16_t' (aka 'unsigned short')
uint16_t
^~~~~~~~
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/_OSByteOrder.h:49:1: note: but in 'DarwinFoundation.OSByteOrder' found different return type '__uint16_t' (aka 'unsigned short')
__uint16_t
^~~~~~~~~~
In module 'Darwin':
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/OSByteOrder.h:24:1: error: '_OSSwapInt32' has different definitions in different modules; definition in module 'Darwin.libkern.OSByteOrder' first difference is return type is 'uint32_t' (aka 'unsigned int')
uint32_t
^~~~~~~~


Welcome to JupyROOT 6.30/02
Applying ATLAS style settings


In [2]:
file_HIST_z = ROOT.TFile.Open("dec20/root_z/HIST.root")
file_HIST_gamma = ROOT.TFile.Open("dec20/root_gamma/HIST.root")

## Efficiencies

In [17]:
def get_HLT_dir(HIST_file, base_dir, keyword):
    file_base_dir = HIST_file.Get(base_dir)
    HLT_dir = []

    keys = file_base_dir.GetListOfKeys()
    for key in keys:
        obj = key.ReadObj()
        if obj.InheritsFrom("TDirectory"):
            name = obj.GetName()
            if keyword in name:
                HLT_dir.append(name)

    return HLT_dir

HLT_dir = get_HLT_dir(HIST_file = file_HIST_z,
                      base_dir = 'run_470000/HLT/TauMon/HLT_Efficiency',
                      keyword = "GNTau")
print(HLT_dir)

Eff_dir = ['HLT_Efficiency_1P', 'HLT_Efficiency_3P']
Eff_plot = ['EffHLT_averageMu_wrt_Offline',
            'EffHLT_tauEta_highPt_wrt_Offline', 'EffHLT_tauEta_wrt_Offline',
            'EffHLT_tauPhi_highPt_wrt_Offline', 'EffHLT_tauPhi_wrt_Offline',
            'EffHLT_tauPt_coarse_wrt_Offline', 'EffHLT_tauPt_wrt_Offline'
            ]

Var_dir = ['HLT_0P', 'HLT_1P', 'HLT_MP']
Var_plot = ['GNTau_TauIDScore', 'GNTau_TauIDScoreSigTrans',
             'Pt', 'Eta', 'Phi',
             'TauVertexX', 'TauVertexY', 'TauVertexZ',
             'averageMu', 'nIsoTrack', 'nTrack']

['HLT_tau100_mediumGNTau_tracktwoMVA_L1eTAU80', 'HLT_tau160_mediumGNTau_tracktwoMVA_L1eTAU140', 'HLT_tau180_mediumGNTau_tracktwoMVA_L1eTAU140', 'HLT_tau200_mediumGNTau_tracktwoMVA_L1eTAU140', 'HLT_tau200_tightGNTau_tracktwoMVA_L1eTAU140', 'HLT_tau20_mediumGNTau_tracktwoMVA_L1cTAU20M', 'HLT_tau25_mediumGNTau_tracktwoMVA_L1cTAU20M', 'HLT_tau30_mediumGNTau_tracktwoMVA_L1cTAU30M', 'HLT_tau35_mediumGNTau_tracktwoMVA_L1cTAU30M', 'HLT_tau40_mediumGNTau_tracktwoMVA_L1cTAU35M', 'HLT_tau60_mediumGNTau_tracktwoMVA_L1eTAU60', 'HLT_tau80_mediumGNTau_tracktwoMVA_L1eTAU80']


In [10]:
def plot_eff_single(path, eff, label, out_dir, ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff.SetLineColor(ROOT.kBlack)
    
    eff.Draw("AP")

    canvas.Update()

    graph = eff.GetPaintedGraph()
    if graph:
        graph.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1-prong'
    elif '3P' in path_parts[5]:
        prong = '3-prong'
    
    legend = ROOT.TLegend(0.19, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + '(' + prong + ')' + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(17)
    legend.SetTextSize(17)
    legend.AddEntry(eff, label, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for hlt_dir in HLT_dir:
    for eff_dir in Eff_dir:
        for eff_plot in Eff_plot:
            path = 'run_470000/HLT/TauMon/HLT_Efficiency' + '/' + hlt_dir + '/' + eff_dir + '/' + eff_plot
            plot_eff_single(path = path,
                            eff = file_HIST_z.Get(path),
                            label = 'mc23_13p6TeV.601191.PhPy8EG_AZNLO_Ztautau',
                            out_dir = 'dec20/eff/z/',
                            ymin = 0,
                            ymax = 1.6
                            )

for hlt_dir in HLT_dir:
    for eff_dir in Eff_dir:
        for eff_plot in Eff_plot:
            path = 'run_470000/HLT/TauMon/HLT_Efficiency' + '/' + hlt_dir + '/' + eff_dir + '/' + eff_plot
            plot_eff_single(path = path,
                            eff = file_HIST_gamma.Get(path),
                            label = 'mc23_13p6TeV.801002.Py8EG_A14NNPDF23LO_Gammatautau_MassWeight',
                            out_dir = 'dec20/eff/gamma/',
                            ymin = 0,
                            ymax = 1.6
                            )

In [11]:
def compare_eff(path, eff_1, eff_2, label_1, label_2, out_dir ,ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff_1.SetLineColor(ROOT.kBlack)
    eff_2.SetLineColor(ROOT.kRed)
    
    eff_1.Draw("AP")
    eff_2.Draw("P SAME")

    canvas.Update()

    graph_1 = eff_1.GetPaintedGraph()
    if graph_1:
        graph_1.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1-prong'
    elif '3P' in path_parts[5]:
        prong = '3-prong'
    
    legend = ROOT.TLegend(0.19, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + '(' + prong + ')' + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(17)
    legend.SetTextSize(17)
    legend.AddEntry(eff_1, label_1, "lp")
    legend.AddEntry(eff_2, label_2, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for hlt_dir in HLT_dir:
    for eff_dir in Eff_dir:
        for eff_plot in Eff_plot:
            path = 'run_470000/HLT/TauMon/HLT_Efficiency' + '/' + hlt_dir + '/' + eff_dir + '/' + eff_plot
            compare_eff(path=path,
                        eff_1=file_HIST_z.Get(path),
                        eff_2=file_HIST_gamma.Get(path),
                        label_1='tau.mc23_13p6TeV.601191.PhPy8EG_AZNLO_Ztautau',
                        label_2='tau.mc23_13p6TeV.801002.Py8EG_A14NNPDF23LO_Gammatautau_MassWeight',
                        out_dir='dec20/eff/compare/',
                        ymin=0,
                        ymax=1.6
                       )

## Distributions

In [28]:
def plot_var_single(path, var, label, out_dir):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    max = var.GetBinContent(var.GetMaximumBin())
    var.SetMaximum(max * 1.5)

    y_axis = var.GetYaxis()
    y_axis.SetTitleOffset(2.2)

    #var.SetLineColor(ROOT.kRed)

    #canvas.SetLogy(1)

    var.Draw()
    canvas.Update()

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/BasicVars/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_1P/Pt'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '0P' in path_parts[5]:
        prong = '0-prong'
    elif '1P' in path_parts[5]:
        prong = '1-prong'
    elif 'MP' in path_parts[5]:
        prong = '3-prong'
    
    legend = ROOT.TLegend(0.19, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + '(' + prong + ')' + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(17)
    legend.SetTextSize(17)
    legend.AddEntry(var, label, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for hlt_dir in HLT_dir:
    for var_dir in Var_dir:
        for var_plot in Var_plot:
            path = 'run_470000/HLT/TauMon/basicVars' + '/' + hlt_dir + '/' + var_dir + '/' + var_plot
            plot_var_single(path = path,
                            var = file_HIST_z.Get(path),
                            label = 'mc23_13p6TeV.601191.PhPy8EG_AZNLO_Ztautau',
                            out_dir = 'dec20/var/z/'
                            )
            
for hlt_dir in HLT_dir:
    for var_dir in Var_dir:
        for var_plot in Var_plot:
            path = 'run_470000/HLT/TauMon/basicVars' + '/' + hlt_dir + '/' + var_dir + '/' + var_plot
            plot_var_single(path = path,
                            var = file_HIST_gamma.Get(path),
                            label = 'mc23_13p6TeV.801002.Py8EG_A14NNPDF23LO_Gammatautau_MassWeight',
                            out_dir = 'dec20/var/gamma/'
                            )

In [33]:
def compare_var(path, var_1, var_2, label_1, label_2, out_dir):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    var_1.Scale(1.0 / var_1.Integral())
    var_2.Scale(1.0 / var_2.Integral())

    max_1 = var_1.GetBinContent(var_1.GetMaximumBin())
    max_2 = var_2.GetBinContent(var_2.GetMaximumBin())
    var_1.SetMaximum(np.max((max_1, max_2)) * 1.7)

    y_axis = var_1.GetYaxis()
    y_axis.SetTitle("A.U. (Normalized)")
    y_axis.SetTitleOffset(2.2)

    var_1.SetLineColor(ROOT.kBlack)
    var_2.SetLineColor(ROOT.kRed)
    var_2.SetMarkerColor(ROOT.kRed)
    
    var_1.Draw()
    var_2.Draw("SAME")

    canvas.Update()

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/basicVars/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_3P/Pt'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '0P' in path_parts[5]:
        prong = '0-prong'
    elif '1P' in path_parts[5]:
        prong = '1-prong'
    elif 'MP' in path_parts[5]:
        prong = '3-prong'
    
    legend = ROOT.TLegend(0.19, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + '(' + prong + ')' + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(17)
    legend.SetTextSize(17)
    legend.AddEntry(var_1, label_1, "lp")
    legend.AddEntry(var_2, label_2, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for hlt_dir in HLT_dir:
    for var_dir in Var_dir:
        for var_plot in Var_plot:
            path = 'run_470000/HLT/TauMon/basicVars' + '/' + hlt_dir + '/' + var_dir + '/' + var_plot
            compare_var(path=path,
                        var_1=file_HIST_z.Get(path),
                        var_2=file_HIST_gamma.Get(path),
                        label_1='tau.mc23_13p6TeV.601191.PhPy8EG_AZNLO_Ztautau',
                        label_2='tau.mc23_13p6TeV.801002.Py8EG_A14NNPDF23LO_Gammatautau_MassWeight',
                        out_dir='dec20/var/compare/'
                       )

## compare releases

In [ ]:
file_HIST_ref = ROOT.TFile.Open("release_68/out.HIST.root")
file_HIST_new = ROOT.TFile.Open("release_72/out.HIST.root")

In [ ]:
base_dir = 'run_470000/HLT/TauMon/HLT_Efficiency'
file_base_dir = file_HIST_ref.Get(base_dir)
HLTpath_dir = []

keys = file_base_dir.GetListOfKeys()
for key in keys:
    obj = key.ReadObj()
    if obj.InheritsFrom("TDirectory"):
        HLTpath_dir.append(obj.GetName())

HLTpath_subdir = ['HLT_Efficiency_1P', 'HLT_Efficiency_3P']
HLTpath_eff = ['EffHLT_averageMu_wrt_Offline',
               'EffHLT_tauEta_highPt_wrt_Offline', 'EffHLT_tauEta_wrt_Offline',
               'EffHLT_tauPhi_highPt_wrt_Offline', 'EffHLT_tauPhi_wrt_Offline',
               'EffHLT_tauPt_coarse_wrt_Offline', 'EffHLT_tauPt_wrt_Offline',
              ]

In [ ]:
def compare_eff(path, eff_ref, eff_new, release_ref, release_new, ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff_ref.SetLineColor(ROOT.kBlack)
    eff_new.SetLineColor(ROOT.kRed)
    
    eff_ref.Draw("AP")
    eff_new.Draw("P SAME")

    canvas.Update()

    graph_ref = eff_ref.GetPaintedGraph()
    if graph_ref:
        graph_ref.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1P'
    elif '3P' in path_parts[5]:
        prong = '3P'
    
    legend = ROOT.TLegend(0.2, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + '[' + prong + ']' + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(17)
    legend.SetTextSize(17)
    legend.AddEntry(eff_ref, release_ref, "lp")
    legend.AddEntry(eff_new, release_new, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = 'plots/' + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for HLT_path in HLTpath_dir:
    for prong in HLTpath_subdir:
        for eff in HLTpath_eff:
            eff_path = base_dir + '/' + HLT_path + '/' + prong + '/' + eff
            compare_eff(eff_path,
                        file_HIST_ref.Get(eff_path),
                        file_HIST_new.Get(eff_path),
                        'Release 24.0.68',
                        'Release 24.0.72',
                        ymin=0,
                        ymax=1.6
                       )